In [1]:
from pathlib import Path
import json
import gzip
import monty.json

In [2]:
with gzip.open(Path("generated", "WyckoffTransformer_mp_20_DiffCSP++.json.gz"), 'rt') as f:
    data = json.load(f)
data = monty.json.MontyDecoder().process_decoded(data)

In [3]:
data[0]

Structure Summary
Lattice
    abc : 5.620519638061523 9.466238021850586 8.957555770874023
 angles : 90.0 90.0 90.0
 volume : 476.58833759282413
      A : 5.620519638061523 0.0 0.0
      B : 0.0 9.466238021850586 0.0
      C : 0.0 0.0 8.957555770874023
    pbc : True True True
PeriodicSite: In (0.0, 1.498, 4.479) [0.0, 0.1582, 0.5]
PeriodicSite: In (0.0, 7.969, 4.479) [0.0, 0.8418, 0.5]
PeriodicSite: In (2.81, 6.231, 0.0) [0.5, 0.6582, 0.0]
PeriodicSite: In (2.81, 3.236, 0.0) [0.5, 0.3418, 0.0]
PeriodicSite: In (2.81, 0.0, 5.926) [0.5, 0.0, 0.6615]
PeriodicSite: In (2.81, 0.0, 3.032) [0.5, 0.0, 0.3385]
PeriodicSite: In (0.0, 4.733, 1.447) [0.0, 0.5, 0.1615]
PeriodicSite: In (0.0, 4.733, 7.511) [0.0, 0.5, 0.8385]
PeriodicSite: In (0.0, 1.759, 0.0) [0.0, 0.1858, 0.0]
PeriodicSite: In (0.0, 7.707, 0.0) [0.0, 0.8142, 0.0]
PeriodicSite: In (2.81, 6.492, 4.479) [0.5, 0.6858, 0.5]
PeriodicSite: In (2.81, 2.974, 4.479) [0.5, 0.3142, 0.5]
PeriodicSite: Cu (0.0, 0.0, 6.932) [0.0, 0.0, 0.7739]
Per